# To Do:
#i have a terrible idea: but wait wtf i dont tuple it later idk why its not working
Figure out Jax
Figure out fun with adagrad/etc
Figure out cnn
AND THEN WHAT I DID THIS ALL FOR! GRADIENT ASCENT BABY

In [41]:
%load_ext autoreload
%autoreload 2
import jax.numpy as jnp
from jax import grad
import jax
import numpy as np
import math
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [128]:
def weight_matrix(shape, naive=False):
    """weight matrix thingy.give dims. Not 0."""
    number = 1
    if(type(shape) == int):
        shape = [shape]
    for i in shape:
        number*= i
    if naive:
        return jnp.array([(i / 10) for i in range(number)]).reshape(*shape)
    return np.array([np.random.uniform(low=-.2, high=.2, size=None) for i in range(number)]).reshape(*shape)
    # return np.array([variable(np.random.uniform(low=-.2, high=.2, size=None)) for i in range(sizes[0] * sizes[1])).reshape(*shape)

In [121]:
    
def sigmoid(x):
    return jnp.vectorize(lambda x: 1/(1+math.e**-x))(x)


def relu(x):
    return jnp.where(x <= 0, 1e-2 * x, x)


In [221]:
   
class Model():
    def __init__(self, input_size, output_size, hidden_layers, naive=False, seed=None):
        '''
        Takes list of # of things in their layers.
        Layers are outputs?
        '''
        if seed != None:
            np.random.seed(seed)

        self.layer_sizes = hidden_layers
        self.layers = []
        self.biases = []
        
        prev_size = input_size
        
        for hidden_layer in hidden_layers:
            self.layers.append(weight_matrix([prev_size, hidden_layer], naive))
            self.biases.append(weight_matrix(hidden_layer, naive))
            prev_size  = hidden_layer
            
        self.biases.append(weight_matrix([output_size]))
        self.layers.append(weight_matrix([prev_size, output_size]))

        self.layers= tuple(self.layers)
        self.biases = tuple(self.biases)  
 
  
    def fd(self, x):
        '''f pass with input. '''

        for i in range(len(self.layers)):
            x = x @ self.layers[i]
            x += self.biases[i]
            if i != len(self.layers) - 1:
                x = relu(x)
            else:
                x = sigmoid(x)
            # self.hidden_states_activation.append(x)

        return x
        
    def loss_static(self, params, x, y):
        '''f pass with for loss.  '''
        w, b = params
        for i in range(len(b)):
            x = x @ w[i]
            x += b[i]
            if i != len(b) - 1:
                x = relu(x)
            else:
                x = sigmoid(x)

        y = jnp.array(y)
        return jnp.sum(x * x - 2 * x * y + y * y))

            
    def train_epoch(self, x, y, lr=10**-2):
        '''
        f pass and then uh gradient descent?

        x:  
        y: the goal. In not sparse tensor.
        lr: how quick it learns
        '''
        losses = []
        x = np.array(x)
        
        for batch_num in range(len(y)):
            mse, grads = jax.value_and_grad(self.loss_static, argnums=(0))((self.layers, self.biases), x[batch_num], y[batch_num])
            
            losses.append(mse)
            
           #0 contains weights and 1 contains the bias grads. 
            # print([i.shape for i in grads[1]])
            # print([i.shape for i in grads[0]])

            #i have a terrible idea: but wait wtf i dont tuple it later idk why its not working
            self.layers = list(self.layers)
            self.biases = list(self.biases)
            
            for i, (layer, grad_layer) in enumerate(zip(self.layers, grads[0])):
                self.layers[i] = layer - lr * grad_layer  
                
            for i, (bias, grad_bias) in enumerate(zip(self.biases, grads[1])):
                self.biases[i] = bias - lr * grad_bias   
                
        preds = self.fd(x[batch_num]) 
        
        correct = jnp.sum(jnp.argmax(preds, axis=1) == jnp.argmax(y[batch_num], axis=1))
        acc = correct / len(y[batch_num])
        print(f"Acc: {acc} Loss: {mse}")
        return losses

In [155]:
test = jnp.arange(5, dtype=float)
test2 = jnp.arange(5, dtype=float)/2
ihatethis(test, test2)

val, grad = jax.value_and_grad(ihatethis)(test, test2)

print(val)
print(grad)

7.5
[0.  0.5 1.  1.5 2. ]


# if this doesnt work ima cry

In [30]:
import tensorflow as tf
import keras
%load_ext autoreload
%autoreload 2

import numpy as np


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data(path="mnist.npz", )
indices = np.arange(len(x_train))
np.random.shuffle(indices)
x_train = x_train[indices]
y_train = y_train[indices]

In [82]:
def batch(x, y, batch_size=32):
    if len(x) % batch_size != 0:
        x = x[:batch_size * (len(x)//batch_size)]
        y=y[:batch_size*(len(x)//batch_size)]
    print(len(x)/batch_size)
    return np.array(np.split(x, int(len(x) / batch_size), axis=0)), np.split(y, int(len(y)/batch_size), axis=0)

def fix_data(x, y):
    x = x.reshape(x.shape[0], 28*28)/255
    test = np.zeros((x.shape[0], 10))
    test[np.arange(x.shape[0]),y] = 1
    return (x, test)

In [85]:
print(b_x.shape)

(125, 4, 784)


In [225]:
fixed_x, fixed_y = fix_data(x_train[:10000], y_train[:10000])
b_x , b_y = batch(fixed_x, fixed_y)


312.0


In [226]:
my_model = Model(28*28, 10, [4, 8])
datas = []
for _epoch in range(10):
    datas.append(my_model.train_epoch(b_x, b_y, lr=10e-1))

Acc: 0.1875 Loss: 0.02851947210729122
Acc: 0.1875 Loss: 0.028401657938957214
Acc: 0.1875 Loss: 0.028073221445083618
Acc: 0.21875 Loss: 0.026867175474762917
Acc: 0.34375 Loss: 0.02428732067346573
Acc: 0.46875 Loss: 0.022608567029237747
Acc: 0.5625 Loss: 0.021389300003647804
Acc: 0.6875 Loss: 0.02021365612745285
Acc: 0.6875 Loss: 0.018730172887444496
Acc: 0.71875 Loss: 0.01743321865797043


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline  

fig = plt.figure(figsize=(10, 7))
pic = 1
for i, img in enumerate(x_test[:10]):
  plt.subplot(2, 5, pic)
  plt.axis('off')
  predicted = my_model.fd(img.flat)
  keras_pred = model.predict(img.reshape(1, 28*28))
  plt.title(f"T {y_test[i]} mine {np.argmax(predicted)} keras {np.argmax(keras_pred)} ")
  plt.imshow(img)
  pic+= 1
plt.show()
#60% acc. Considering this is from nearly scratch not terrible 